In [ ]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import symfem

In [ ]:
p_1, p_2, p_3 = sympy.symbols('p_1 p_2 p_3')
p_1_x, p_2_x, p_1_y, p_2_y = sympy.symbols('p_1_x p_2_x p_1_y p_2_y')
x, y = sympy.symbols("x, y")

In [ ]:
all_basis = symfem.create_element("triangle", "Lagrange", 2).get_basis_functions()
all_basis = [itm.subs(x, p_1).subs(y, p_2) for itm in all_basis]

In [ ]:
result = {}
args = [p_1_x, p_2_x, p_1_y, p_2_y]

J_inv_det = sympy.Matrix(args).reshape(2, 2).inv().det()

for i in tqdm(range(6)):
    for j in range(6):
        
        first = all_basis[i].subs(p_3, 1-p_1-p_2)
        second = all_basis[j].subs(p_3, 1-p_1-p_2)
        
        u_x = first.diff(p_1)*p_1_x + first.diff(p_2)*p_2_x
        u_y = first.diff(p_1)*p_1_y + first.diff(p_2)*p_2_y

        v_x = second.diff(p_1)*p_1_x + second.diff(p_2)*p_2_x
        v_y = second.diff(p_1)*p_1_y + second.diff(p_2)*p_2_y

        integral = sympy.integrate(u_x*v_x + u_y*v_y, (p_2, 0, 1-p_1), (p_1, 0, 1)) * J_inv_det
        
        func = sympy.lambdify(args, integral)
        result[f'{i}_{j}'] = func

In [ ]:
for idx, basis in enumerate(all_basis):
    basis_norm = basis.subs(p_3, 1-p_1-p_2)
    sym_int = sympy.integrate(basis_norm * J_inv_det, (p_2, 0, 1-p_1), (p_1, 0, 1))    
    result[f'{idx}'] = sympy.lambdify(args, sym_int)

In [ ]:
import dill
dill.settings['recurse'] = True
dill.dump(result, open('../calculations/poisson_quad', 'wb'))